# Fin du prétraitement : des inputs de même longueur et prêts pour le CNN

Pour pouvoir exécuter notre réseau de neurone (CNN), nous avons besoin de traiter encore nos données. 

En effet, notre modèle va apprendre les caractéristiques générales de plusieurs vols entre deux évenèments (WW ou SV), pour chaque avion. Cependant, pour que nous puissions effectuer un réseau de neurones, nous avons besoin d'avoir des intervalles "de même longueur", c'est à dire avec le même nombre de vols pour chaque intervalles. Pour l'instant, certains avions ont plus de 9000 vols entre deux évènements tandis que d'autres en ont très peu, et c'est pourquoi nous devons les traiter.

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [20]:
#Importation des données pré-traitées par Louise
path_df = r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Safran\pretraitement.csv'
safran=pd.read_csv(path_df ,sep=',', encoding='latin-1')

In [21]:
safran

,date,Unnamed: 0,engine_serial_number,engine_family,engine_series,cycles,cycles_counter,egt_margin,var_mot_1,flight_leg_hours,...,Interpolate_flight_leg_hours,Interpolate_SV_rank,Interpolate_Config_B_rank,Interpolate_WW_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,Interpolate_egt_slope
0,2019-04-29 06:29:58,1,ESN_1,Engine_family_1,Engine_series_1,14.699402,14,0.881646,-0.313549,0.857778,...,0.857778,0.0,0.0,0.0,-0.261068,0.193871,0.448627,0.0,0.601803,-0.029193
1,2019-04-29 08:10:00,2,ESN_1,Engine_family_1,Engine_series_1,15.284274,15,0.792029,0.006330,0.794167,...,0.794167,0.0,0.0,0.0,-0.064202,0.273855,1.500848,0.0,-1.056965,-0.029193
2,2019-04-29 09:55:00,3,ESN_1,Engine_family_1,Engine_series_1,15.898185,16,0.706729,-0.286324,0.736667,...,0.736667,0.0,0.0,0.0,-0.292673,0.193871,0.764293,0.0,0.149412,-0.029193
3,2019-04-29 11:36:53,4,ESN_1,Engine_family_1,Engine_series_1,16.493874,17,0.702078,0.430174,0.802500,...,0.802500,0.0,0.0,0.0,0.070056,0.273855,1.500848,0.0,-1.056965,-0.029193
4,2019-04-30 04:28:40,5,ESN_1,Engine_family_1,Engine_series_1,22.409543,18,0.645941,0.299420,0.817500,...,0.817500,0.0,0.0,0.0,-0.463185,0.193871,0.448627,0.0,0.601803,-0.029193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450270,2022-10-26 07:39:15,2911298,ESN_1369,Engine_family_1,Engine_series_6,34.993550,50,0.531868,-0.731730,2.654444,...,2.654444,0.0,0.0,0.0,0.728625,-0.356159,-0.708817,0.0,-1.207762,0.000968
2450271,2022-10-26 11:36:39,2911299,ESN_1369,Engine_family_1,Engine_series_6,35.190820,51,0.973045,0.364383,2.501667,...,2.501667,0.0,0.0,0.0,0.421328,-0.351647,-0.077484,0.0,0.199678,0.000968
2450272,2022-10-27 03:55:34,2911301,ESN_1369,Engine_family_1,Engine_series_6,36.001950,53,0.800778,0.949444,2.165000,...,2.165000,0.0,0.0,0.0,1.092857,-0.351647,0.343405,0.0,-0.403511,0.000968
2450273,2022-10-27 07:33:12,2911302,ESN_1369,Engine_family_1,Engine_series_6,36.182090,54,0.619281,-0.748008,2.536667,...,2.536667,0.0,0.0,0.0,1.383080,-0.356159,-0.708817,0.0,-0.705105,0.000968


In [22]:
safran.columns

Index(['date', 'Unnamed: 0', 'engine_serial_number', 'engine_family',
       'engine_series', 'cycles', 'cycles_counter', 'egt_margin', 'var_mot_1',
       'flight_leg_hours', 'event_rank', 'egt_slope', 'SV_indicator',
       'SV_rank', 'Config_B_indicator', 'Config_B_rank', 'WW_indicator',
       'WW_rank', 'config_A', 'config_B', 'var_env_1', 'var_env_2',
       'var_env_3', 'var_env_4', 'var_env_5', 'Interpolate_egt_margin',
       'Interpolate_var_mot_1', 'Interpolate_flight_leg_hours',
       'Interpolate_SV_rank', 'Interpolate_Config_B_rank',
       'Interpolate_WW_rank', 'Interpolate_var_env_1', 'Interpolate_var_env_2',
       'Interpolate_var_env_3', 'Interpolate_var_env_4',
       'Interpolate_var_env_5', 'Interpolate_egt_slope'],
      dtype='object')

In [23]:
#Pour plus de clarté, je ne garde que les colonnes qui ont déja été travaillées et celles nécessaires au traitement des intervalles
safran_2 = safran[['date', "engine_serial_number",'engine_series', 'cycles', 'cycles_counter','Interpolate_egt_margin',
       'Interpolate_var_mot_1','event_rank', 'config_A', 'config_B','Interpolate_flight_leg_hours',
       'Interpolate_SV_rank', 'Interpolate_Config_B_rank',
       'Interpolate_WW_rank', 'Interpolate_var_env_1', 'Interpolate_var_env_2',
       'Interpolate_var_env_3', 'Interpolate_var_env_4',
       'Interpolate_var_env_5', 'Interpolate_egt_slope']]


In [24]:
safran_2

,date,engine_serial_number,engine_series,cycles,cycles_counter,Interpolate_egt_margin,Interpolate_var_mot_1,event_rank,config_A,config_B,Interpolate_flight_leg_hours,Interpolate_SV_rank,Interpolate_Config_B_rank,Interpolate_WW_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,Interpolate_egt_slope
0,2019-04-29 06:29:58,ESN_1,Engine_series_1,14.699402,14,0.881646,-0.313549,0,Config_A_1,Config_B_1,0.857778,0.0,0.0,0.0,-0.261068,0.193871,0.448627,0.0,0.601803,-0.029193
1,2019-04-29 08:10:00,ESN_1,Engine_series_1,15.284274,15,0.792029,0.006330,0,Config_A_1,Config_B_1,0.794167,0.0,0.0,0.0,-0.064202,0.273855,1.500848,0.0,-1.056965,-0.029193
2,2019-04-29 09:55:00,ESN_1,Engine_series_1,15.898185,16,0.706729,-0.286324,0,Config_A_1,Config_B_1,0.736667,0.0,0.0,0.0,-0.292673,0.193871,0.764293,0.0,0.149412,-0.029193
3,2019-04-29 11:36:53,ESN_1,Engine_series_1,16.493874,17,0.702078,0.430174,0,Config_A_1,Config_B_1,0.802500,0.0,0.0,0.0,0.070056,0.273855,1.500848,0.0,-1.056965,-0.029193
4,2019-04-30 04:28:40,ESN_1,Engine_series_1,22.409543,18,0.645941,0.299420,0,Config_A_1,Config_B_1,0.817500,0.0,0.0,0.0,-0.463185,0.193871,0.448627,0.0,0.601803,-0.029193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450270,2022-10-26 07:39:15,ESN_1369,Engine_series_6,34.993550,50,0.531868,-0.731730,0,Config_A_3,Config_B_1,2.654444,0.0,0.0,0.0,0.728625,-0.356159,-0.708817,0.0,-1.207762,0.000968
2450271,2022-10-26 11:36:39,ESN_1369,Engine_series_6,35.190820,51,0.973045,0.364383,0,Config_A_3,Config_B_1,2.501667,0.0,0.0,0.0,0.421328,-0.351647,-0.077484,0.0,0.199678,0.000968
2450272,2022-10-27 03:55:34,ESN_1369,Engine_series_6,36.001950,53,0.800778,0.949444,0,Config_A_3,Config_B_1,2.165000,0.0,0.0,0.0,1.092857,-0.351647,0.343405,0.0,-0.403511,0.000968
2450273,2022-10-27 07:33:12,ESN_1369,Engine_series_6,36.182090,54,0.619281,-0.748008,0,Config_A_3,Config_B_1,2.536667,0.0,0.0,0.0,1.383080,-0.356159,-0.708817,0.0,-0.705105,0.000968


## 1) Choix de la taille de l'intervalle

On utilise la variable **event_rank** pour identifier tous les intervalles pendant lequel aucun évènement n'a été effectué.

In [25]:
safran_ER = safran_2[["engine_serial_number", "event_rank", "Interpolate_egt_slope"]]

In [26]:
# On a un .count(), donc on peut prend n'importe quelle colonne pour vérifier combien il y a de vols
safran_ER.groupby(by=["engine_serial_number", "event_rank"]).count()

Interpolate_egt_slope
engine_serial_number event_rank                       
ESN_1                0                            4548
                     1                             644
                     2                             468
                     3                             380
                     4                            2200
...                                                ...
ESN_998              1                             140
                     2                              32
ESN_999              0                             466
                     1                             444
                     2                             162

[8439 rows x 1 columns]

- **Intervalles avec le moins de données**


Regardons maintenant les cas où l'on a le moins de données pour un même intervalle.

In [27]:
safran_ER.groupby(by=["engine_serial_number", "event_rank"]).count().sort_values(by="Interpolate_egt_slope").head(220)

,,Interpolate_egt_slope
engine_serial_number,event_rank,
ESN_454,5,1
ESN_255,14,1
ESN_35,15,1
ESN_32,17,1
ESN_653,6,1
...,...,...
ESN_1243,1,10
ESN_181,3,10
ESN_137,7,10


In [28]:
saf_ER_group = safran_ER.groupby(by=["engine_serial_number", "event_rank"]).count().sort_values(by="Interpolate_egt_slope")
saf_ER_group = saf_ER_group.rename(columns = {'Interpolate_egt_slope': 'Nb_vols_entre_event'})

#On a encore engine_serial_number et Interpolate_WW_rank en index, on les enlève
saf_ER_group = saf_ER_group.reset_index()

#On réindexe, pour avoir une colonne avec les identifiants de chaque intervalle, 
#qu'on appelle id_int pour identifiant intervalle
saf_ER_group['id_int'] = saf_ER_group.index
saf_ER_group.sort_values(by="Nb_vols_entre_event")
saf_ER_group

,engine_serial_number,event_rank,Nb_vols_entre_event,id_int
0,ESN_454,5,1,0
1,ESN_255,14,1,1
2,ESN_35,15,1,2
3,ESN_32,17,1,3
4,ESN_653,6,1,4
...,...,...,...,...
8434,ESN_1,4,2200,8434
8435,ESN_19,0,2241,8435
8436,ESN_250,8,2275,8436
8437,ESN_1,10,3568,8437


In [29]:
#De cette manière, on peut visualiser les intervalles où le nombre de vols est inférieur à un certain seuil
saf_ER_group[(saf_ER_group.Nb_vols_entre_event < 100)]

,engine_serial_number,event_rank,Nb_vols_entre_event,id_int
0,ESN_454,5,1,0
1,ESN_255,14,1,1
2,ESN_35,15,1,2
3,ESN_32,17,1,3
4,ESN_653,6,1,4
...,...,...,...,...
1818,ESN_165,12,99,1818
1819,ESN_146,4,99,1819
1820,ESN_59,2,99,1820
1821,ESN_44,2,99,1821


In [30]:
print("Le nombre d'intervalles de temps avec moins de 25 vols correspond à", round((saf_ER_group[(saf_ER_group.Nb_vols_entre_event < 25)].shape[0]/saf_ER_group.shape[0])*100,2), "% des intervalles")
print("Le nombre d'intervalles de temps avec moins de 50 vols correspond à", round((saf_ER_group[(saf_ER_group.Nb_vols_entre_event < 50)].shape[0]/saf_ER_group.shape[0])*100,2), "% des intervalles")
print("Le nombre d'intervalles de temps avec moins de 100 vols correspond à", round((saf_ER_group[(saf_ER_group.Nb_vols_entre_event < 100)].shape[0]/saf_ER_group.shape[0])*100,2), "% des intervalles")
print("Le nombre d'intervalles de temps avec moins de 150 vols correspond à", round((saf_ER_group[(saf_ER_group.Nb_vols_entre_event < 150)].shape[0]/saf_ER_group.shape[0])*100,2), "% des intervalles")

Le nombre d'intervalles de temps avec moins de 25 vols correspond à 5.18 % des intervalles
Le nombre d'intervalles de temps avec moins de 50 vols correspond à 9.93 % des intervalles
Le nombre d'intervalles de temps avec moins de 100 vols correspond à 21.6 % des intervalles
Le nombre d'intervalles de temps avec moins de 150 vols correspond à 30.93 % des intervalles


In [31]:
#On merge pour avoir l'identifiant dans la base de données
safran_complete = pd.merge(safran_2, saf_ER_group, left_on = ["engine_serial_number", "event_rank"], right_on = ["engine_serial_number", "event_rank"], how="inner")
safran_complete

,date,engine_serial_number,engine_series,cycles,cycles_counter,Interpolate_egt_margin,Interpolate_var_mot_1,event_rank,config_A,config_B,...,Interpolate_Config_B_rank,Interpolate_WW_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,Interpolate_egt_slope,Nb_vols_entre_event,id_int
0,2019-04-29 06:29:58,ESN_1,Engine_series_1,14.699402,14,0.881646,-0.313549,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.261068,0.193871,0.448627,0.0,0.601803,-0.029193,4548,8438
1,2019-04-29 08:10:00,ESN_1,Engine_series_1,15.284274,15,0.792029,0.006330,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.064202,0.273855,1.500848,0.0,-1.056965,-0.029193,4548,8438
2,2019-04-29 09:55:00,ESN_1,Engine_series_1,15.898185,16,0.706729,-0.286324,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.292673,0.193871,0.764293,0.0,0.149412,-0.029193,4548,8438
3,2019-04-29 11:36:53,ESN_1,Engine_series_1,16.493874,17,0.702078,0.430174,0,Config_A_1,Config_B_1,...,0.0,0.0,0.070056,0.273855,1.500848,0.0,-1.056965,-0.029193,4548,8438
4,2019-04-30 04:28:40,ESN_1,Engine_series_1,22.409543,18,0.645941,0.299420,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.463185,0.193871,0.448627,0.0,0.601803,-0.029193,4548,8438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450270,2022-10-26 07:39:15,ESN_1369,Engine_series_6,34.993550,50,0.531868,-0.731730,0,Config_A_3,Config_B_1,...,0.0,0.0,0.728625,-0.356159,-0.708817,0.0,-1.207762,0.000968,34,576
2450271,2022-10-26 11:36:39,ESN_1369,Engine_series_6,35.190820,51,0.973045,0.364383,0,Config_A_3,Config_B_1,...,0.0,0.0,0.421328,-0.351647,-0.077484,0.0,0.199678,0.000968,34,576
2450272,2022-10-27 03:55:34,ESN_1369,Engine_series_6,36.001950,53,0.800778,0.949444,0,Config_A_3,Config_B_1,...,0.0,0.0,1.092857,-0.351647,0.343405,0.0,-0.403511,0.000968,34,576
2450273,2022-10-27 07:33:12,ESN_1369,Engine_series_6,36.182090,54,0.619281,-0.748008,0,Config_A_3,Config_B_1,...,0.0,0.0,1.383080,-0.356159,-0.708817,0.0,-0.705105,0.000968,34,576


### 2) L'échantillonnage avec 100 vols par intervalles

Après avoir réfléchi pour conserver à al fois beaucoup d'intervalles différents, mais également le plus de vols possibles au sein de ces intervalles, , on a choisit de conserver les intervalles avec plus de 100 vols.

In [32]:
df_mauvais_100 = safran_complete[safran_complete["Nb_vols_entre_event"].between(0, 99)] # Les vols qu'on supprime
df_keep_100 = safran_complete[~safran_complete["Nb_vols_entre_event"].between(0,99)] #Les vols qu'on garde
df_keep_100
#On conserve uniquement les intervalles où on a plus de 100 dans df_keep_100 données

,date,engine_serial_number,engine_series,cycles,cycles_counter,Interpolate_egt_margin,Interpolate_var_mot_1,event_rank,config_A,config_B,...,Interpolate_Config_B_rank,Interpolate_WW_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,Interpolate_egt_slope,Nb_vols_entre_event,id_int
0,2019-04-29 06:29:58,ESN_1,Engine_series_1,14.699402,14,0.881646,-0.313549,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.261068,0.193871,0.448627,0.0,0.601803,-0.029193,4548,8438
1,2019-04-29 08:10:00,ESN_1,Engine_series_1,15.284274,15,0.792029,0.006330,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.064202,0.273855,1.500848,0.0,-1.056965,-0.029193,4548,8438
2,2019-04-29 09:55:00,ESN_1,Engine_series_1,15.898185,16,0.706729,-0.286324,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.292673,0.193871,0.764293,0.0,0.149412,-0.029193,4548,8438
3,2019-04-29 11:36:53,ESN_1,Engine_series_1,16.493874,17,0.702078,0.430174,0,Config_A_1,Config_B_1,...,0.0,0.0,0.070056,0.273855,1.500848,0.0,-1.056965,-0.029193,4548,8438
4,2019-04-30 04:28:40,ESN_1,Engine_series_1,22.409543,18,0.645941,0.299420,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.463185,0.193871,0.448627,0.0,0.601803,-0.029193,4548,8438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450219,2022-10-27 03:49:41,ESN_1360,Engine_series_1,254.604500,274,0.919735,0.869409,0,Config_A_3,Config_B_1,...,0.0,0.0,0.064519,-0.368762,0.238182,0.0,0.451006,-0.004884,210,3443
2450220,2022-10-27 12:43:08,ESN_1360,Engine_series_1,256.982600,275,0.686130,0.279454,0,Config_A_3,Config_B_1,...,0.0,0.0,-0.223848,-0.311284,-0.919261,0.0,-1.056965,-0.004884,210,3443
2450221,2022-10-27 15:30:42,ESN_1360,Engine_series_1,257.729600,276,0.721306,0.346712,0,Config_A_3,Config_B_1,...,0.0,0.0,-0.334779,-0.303992,-1.971483,0.0,0.551538,-0.004884,210,3443
2450222,2022-10-27 18:33:59,ESN_1360,Engine_series_1,258.546700,277,0.675019,0.313983,0,Config_A_3,Config_B_1,...,0.0,0.0,1.174560,-0.311284,0.132960,0.0,-2.363873,-0.004884,210,3443


In [33]:
#ATTENTION cette cellule peut prendre 2-3 minutes à s'éxécuter
df_ech_100 = df_keep_100.groupby("id_int").sample(100) #On utilise sample(100) pour avoir 100 données aléatoires par intervalle
df_ech_100

,date,engine_serial_number,engine_series,cycles,cycles_counter,Interpolate_egt_margin,Interpolate_var_mot_1,event_rank,config_A,config_B,...,Interpolate_Config_B_rank,Interpolate_WW_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,Interpolate_egt_slope,Nb_vols_entre_event,id_int
568975,2022-10-11 03:31:43,ESN_139,Engine_series_1,4027.205000,3680,-2.330820,-1.164945,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.553237,-0.363494,-1.037065,0.0,1.099241,-0.027761,100,1823
568990,2022-10-14 14:54:58,ESN_139,Engine_series_1,4044.802000,3696,-2.457672,-0.928283,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.461113,-0.318391,0.448627,0.0,-0.755371,-0.027761,100,1823
569026,2022-10-20 11:49:15,ESN_139,Engine_series_1,4103.547000,3735,-2.318243,-0.599568,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.505691,-0.348121,-0.417907,0.0,-1.177604,-0.027761,100,1823
569038,2022-10-22 15:20:56,ESN_139,Engine_series_1,4123.906000,3748,-2.248718,-0.483088,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.514615,-0.190762,-0.708817,0.0,-0.101916,-0.027761,100,1823
568998,2022-10-15 21:56:13,ESN_139,Engine_series_1,4055.000000,3704,-2.466506,-0.847961,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.595453,-0.190762,-1.340150,0.0,1.456320,-0.027761,100,1823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,2019-05-26 19:32:52,ESN_1,Engine_series_1,249.771155,256,0.444726,0.610305,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.423066,0.273855,0.553849,0.0,0.149412,-0.029193,4548,8438
3933,2019-06-29 15:40:43,ESN_1,Engine_series_1,533.452827,543,0.028035,0.680037,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.556554,0.637822,0.343405,0.0,-1.056965,-0.029193,4548,8438
783,2019-07-30 17:04:56,ESN_1,Engine_series_1,795.220369,814,-0.253356,0.366438,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.549613,0.056626,-1.024483,0.0,-0.855902,-0.029193,4548,8438
2238,2019-10-08 06:09:29,ESN_1,Engine_series_1,1155.621213,1157,-0.849156,-0.291145,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.254914,0.159720,-1.655816,0.0,0.802866,-0.029193,4548,8438


In [34]:
print("En effectuant cette méthode, on aura", df_ech_100.shape[0], "lignes dans notre base de données, alors que la base contenait", safran_2.shape[0], "lignes.")

En effectuant cette méthode, on aura 661600 lignes dans notre base de données, alors que la base contenait 2450275 lignes.


In [35]:
df_ech_100.sort_values(by=["id_int", "Interpolate_egt_slope"])

,date,engine_serial_number,engine_series,cycles,cycles_counter,Interpolate_egt_margin,Interpolate_var_mot_1,event_rank,config_A,config_B,...,Interpolate_Config_B_rank,Interpolate_WW_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,Interpolate_egt_slope,Nb_vols_entre_event,id_int
568975,2022-10-11 03:31:43,ESN_139,Engine_series_1,4027.205000,3680,-2.330820,-1.164945,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.553237,-0.363494,-1.037065,0.0,1.099241,-0.027761,100,1823
568990,2022-10-14 14:54:58,ESN_139,Engine_series_1,4044.802000,3696,-2.457672,-0.928283,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.461113,-0.318391,0.448627,0.0,-0.755371,-0.027761,100,1823
569026,2022-10-20 11:49:15,ESN_139,Engine_series_1,4103.547000,3735,-2.318243,-0.599568,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.505691,-0.348121,-0.417907,0.0,-1.177604,-0.027761,100,1823
569038,2022-10-22 15:20:56,ESN_139,Engine_series_1,4123.906000,3748,-2.248718,-0.483088,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.514615,-0.190762,-0.708817,0.0,-0.101916,-0.027761,100,1823
568998,2022-10-15 21:56:13,ESN_139,Engine_series_1,4055.000000,3704,-2.466506,-0.847961,14,Config_A_2,Config_B_4,...,6.0,8.0,-0.595453,-0.190762,-1.340150,0.0,1.456320,-0.027761,100,1823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,2019-05-26 19:32:52,ESN_1,Engine_series_1,249.771155,256,0.444726,0.610305,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.423066,0.273855,0.553849,0.0,0.149412,-0.029193,4548,8438
3933,2019-06-29 15:40:43,ESN_1,Engine_series_1,533.452827,543,0.028035,0.680037,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.556554,0.637822,0.343405,0.0,-1.056965,-0.029193,4548,8438
783,2019-07-30 17:04:56,ESN_1,Engine_series_1,795.220369,814,-0.253356,0.366438,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.549613,0.056626,-1.024483,0.0,-0.855902,-0.029193,4548,8438
2238,2019-10-08 06:09:29,ESN_1,Engine_series_1,1155.621213,1157,-0.849156,-0.291145,0,Config_A_1,Config_B_1,...,0.0,0.0,-0.254914,0.159720,-1.655816,0.0,0.802866,-0.029193,4548,8438


### 3) Les derniers traitements numériques

Enfin, nous choisissons quelles varibles éliminer : 
- **engine_family** car dans notre base de donnés, tous les moteurs d'avions sont issus de la même famille de moteur
- **date** : car les informations sur la temporalité sont contenues dans cycle_counter
- **cycle** : pour la même raison que la variable "date"
- **event_rank** : car les informations que nous souhaitions ont déjà servi pour constituer la variable id_int qui identifie tous les intervalles

In [36]:
Y = df_ech_100[["Interpolate_egt_slope", "id_int"]]
X = df_ech_100[["engine_serial_number", "engine_series",
                "cycles_counter",'config_A', 'config_B', "Interpolate_var_mot_1",
                'Interpolate_flight_leg_hours', 'Interpolate_SV_rank','Interpolate_WW_rank',
                'Interpolate_Config_B_rank', 'Interpolate_var_env_1',
                'Interpolate_var_env_2','Interpolate_var_env_3', 
                'Interpolate_var_env_4','Interpolate_var_env_5', "id_int"]] # Cas où on a besoin d'un seul dataframe

In [37]:
X

,engine_serial_number,engine_series,cycles_counter,config_A,config_B,Interpolate_var_mot_1,Interpolate_flight_leg_hours,Interpolate_SV_rank,Interpolate_WW_rank,Interpolate_Config_B_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,id_int
568975,ESN_139,Engine_series_1,3680,Config_A_2,Config_B_4,-1.164945,1.278333,1.0,8.0,6.0,-0.553237,-0.363494,-1.037065,0.0,1.099241,1823
568990,ESN_139,Engine_series_1,3696,Config_A_2,Config_B_4,-0.928283,0.961944,1.0,8.0,6.0,-0.461113,-0.318391,0.448627,0.0,-0.755371,1823
569026,ESN_139,Engine_series_1,3735,Config_A_2,Config_B_4,-0.599568,1.945278,1.0,8.0,6.0,-0.505691,-0.348121,-0.417907,0.0,-1.177604,1823
569038,ESN_139,Engine_series_1,3748,Config_A_2,Config_B_4,-0.483088,1.776111,1.0,8.0,6.0,-0.514615,-0.190762,-0.708817,0.0,-0.101916,1823
568998,ESN_139,Engine_series_1,3704,Config_A_2,Config_B_4,-0.847961,1.631944,1.0,8.0,6.0,-0.595453,-0.190762,-1.340150,0.0,1.456320,1823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,ESN_1,Engine_series_1,256,Config_A_1,Config_B_1,0.610305,0.899722,0.0,0.0,0.0,-0.423066,0.273855,0.553849,0.0,0.149412,8438
3933,ESN_1,Engine_series_1,543,Config_A_1,Config_B_1,0.680037,1.520000,0.0,0.0,0.0,-0.556554,0.637822,0.343405,0.0,-1.056965,8438
783,ESN_1,Engine_series_1,814,Config_A_1,Config_B_1,0.366438,0.763333,0.0,0.0,0.0,-0.549613,0.056626,-1.024483,0.0,-0.855902,8438
2238,ESN_1,Engine_series_1,1157,Config_A_1,Config_B_1,-0.291145,0.796944,0.0,0.0,0.0,-0.254914,0.159720,-1.655816,0.0,0.802866,8438


In [38]:
X_float=X.copy()

In [39]:
X_float.loc[:,"engine_serial_number"] = X['engine_serial_number'].str[4:]
X_float.loc[:,'engine_serial_number'] = X['engine_serial_number'].str[4:]
X_float.loc[:,'engine_series'] = X['engine_series'].str[14:]
X_float.loc[:,'config_A'] = X['config_A'].str[9:]
X_float.loc[:,'config_B'] = X['config_B'].str[9:]
X_float= X_float.astype('float')

In [40]:
X_float

,engine_serial_number,engine_series,cycles_counter,config_A,config_B,Interpolate_var_mot_1,Interpolate_flight_leg_hours,Interpolate_SV_rank,Interpolate_WW_rank,Interpolate_Config_B_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,id_int
568975,139.0,1.0,3680.0,2.0,4.0,-1.164945,1.278333,1.0,8.0,6.0,-0.553237,-0.363494,-1.037065,0.0,1.099241,1823.0
568990,139.0,1.0,3696.0,2.0,4.0,-0.928283,0.961944,1.0,8.0,6.0,-0.461113,-0.318391,0.448627,0.0,-0.755371,1823.0
569026,139.0,1.0,3735.0,2.0,4.0,-0.599568,1.945278,1.0,8.0,6.0,-0.505691,-0.348121,-0.417907,0.0,-1.177604,1823.0
569038,139.0,1.0,3748.0,2.0,4.0,-0.483088,1.776111,1.0,8.0,6.0,-0.514615,-0.190762,-0.708817,0.0,-0.101916,1823.0
568998,139.0,1.0,3704.0,2.0,4.0,-0.847961,1.631944,1.0,8.0,6.0,-0.595453,-0.190762,-1.340150,0.0,1.456320,1823.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,1.0,1.0,256.0,1.0,1.0,0.610305,0.899722,0.0,0.0,0.0,-0.423066,0.273855,0.553849,0.0,0.149412,8438.0
3933,1.0,1.0,543.0,1.0,1.0,0.680037,1.520000,0.0,0.0,0.0,-0.556554,0.637822,0.343405,0.0,-1.056965,8438.0
783,1.0,1.0,814.0,1.0,1.0,0.366438,0.763333,0.0,0.0,0.0,-0.549613,0.056626,-1.024483,0.0,-0.855902,8438.0
2238,1.0,1.0,1157.0,1.0,1.0,-0.291145,0.796944,0.0,0.0,0.0,-0.254914,0.159720,-1.655816,0.0,0.802866,8438.0


In [41]:
# On normalise toutes les colonnes pour ne pas biaiser l'apprentissage de notre réseau de neurones
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scale = X_float.copy()

X_scale[['engine_serial_number','engine_series','cycles_counter',
                 'config_A','config_B',"Interpolate_var_mot_1",'Interpolate_flight_leg_hours',
                 'Interpolate_SV_rank','Interpolate_Config_B_rank',
                 'Interpolate_var_env_1','Interpolate_var_env_2',
                 'Interpolate_var_env_3','Interpolate_var_env_4','Interpolate_var_env_5']] = scaler.fit_transform(
    X_scale[['engine_serial_number','engine_series','cycles_counter',
                 'config_A','config_B',"Interpolate_var_mot_1",'Interpolate_flight_leg_hours',
                 'Interpolate_SV_rank','Interpolate_Config_B_rank',
                 'Interpolate_var_env_1','Interpolate_var_env_2',
                 'Interpolate_var_env_3','Interpolate_var_env_4','Interpolate_var_env_5']])

X_scale

,engine_serial_number,engine_series,cycles_counter,config_A,config_B,Interpolate_var_mot_1,Interpolate_flight_leg_hours,Interpolate_SV_rank,Interpolate_WW_rank,Interpolate_Config_B_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,id_int
568975,0.101545,0.0,0.479729,0.333333,1.0,0.323262,0.123242,0.333333,8.0,0.6,0.001248,0.000382,0.469181,0.0,0.184318,1823.0
568990,0.101545,0.0,0.481815,0.333333,1.0,0.361258,0.101395,0.333333,8.0,0.6,0.003273,0.002476,0.635294,0.0,0.111111,1823.0
569026,0.101545,0.0,0.486899,0.333333,1.0,0.414033,0.169298,0.333333,8.0,0.6,0.002293,0.001096,0.538408,0.0,0.094444,1823.0
569038,0.101545,0.0,0.488593,0.333333,1.0,0.432734,0.157616,0.333333,8.0,0.6,0.002097,0.008403,0.505882,0.0,0.136905,1823.0
568998,0.101545,0.0,0.482858,0.333333,1.0,0.374153,0.147661,0.333333,8.0,0.6,0.000319,0.008403,0.435294,0.0,0.198413,1823.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,0.000000,0.0,0.033372,0.000000,0.0,0.608278,0.097098,0.000000,0.0,0.0,0.004110,0.029978,0.647059,0.0,0.146825,8438.0
3933,0.000000,0.0,0.070786,0.000000,0.0,0.619473,0.139931,0.000000,0.0,0.0,0.001175,0.046879,0.623529,0.0,0.099206,8438.0
783,0.000000,0.0,0.106114,0.000000,0.0,0.569125,0.087680,0.000000,0.0,0.0,0.001327,0.019891,0.470588,0.0,0.107143,8438.0
2238,0.000000,0.0,0.150828,0.000000,0.0,0.463550,0.090001,0.000000,0.0,0.0,0.007807,0.024678,0.400000,0.0,0.172619,8438.0


In [42]:
Y_doublon = Y.copy()

In [43]:
Y_doublon.drop_duplicates(subset = "id_int",keep = 'first', inplace=True)

In [44]:
Y_doublon

,Interpolate_egt_slope,id_int
568975,-0.027761,1823
2358357,-0.025889,1824
1910477,-0.034384,1825
225270,-0.019224,1826
2345000,-0.047450,1827
...,...,...
6554,-0.010711,8434
103741,-0.004308,8435
927016,-0.061415,8436
14999,-0.006345,8437


On a 6616 lignes, donc aucun doublon. Les bases peuvent être téléchargées.

In [ ]:
#On télécharge nos base X et Y en csv
X_scale.to_csv('D:/Données/ENSAE/2A/S2/Séminaire de modélisation statistique/X.csv')
Y.to_csv('D:/Données/ENSAE/2A/S2/Séminaire de modélisation statistique/Y.csv')

In [46]:
# On télécharge également la base complete composée de X et de Y.
X_Y = pd.merge(X_scale,Y_doublon)

In [47]:
X_Y

,engine_serial_number,engine_series,cycles_counter,config_A,config_B,Interpolate_var_mot_1,Interpolate_flight_leg_hours,Interpolate_SV_rank,Interpolate_WW_rank,Interpolate_Config_B_rank,Interpolate_var_env_1,Interpolate_var_env_2,Interpolate_var_env_3,Interpolate_var_env_4,Interpolate_var_env_5,id_int,Interpolate_egt_slope
0,0.101545,0.0,0.479729,0.333333,1.0,0.323262,0.123242,0.333333,8.0,0.6,0.001248,0.000382,0.469181,0.0,0.184318,1823.0,-0.027761
1,0.101545,0.0,0.481815,0.333333,1.0,0.361258,0.101395,0.333333,8.0,0.6,0.003273,0.002476,0.635294,0.0,0.111111,1823.0,-0.027761
2,0.101545,0.0,0.486899,0.333333,1.0,0.414033,0.169298,0.333333,8.0,0.6,0.002293,0.001096,0.538408,0.0,0.094444,1823.0,-0.027761
3,0.101545,0.0,0.488593,0.333333,1.0,0.432734,0.157616,0.333333,8.0,0.6,0.002097,0.008403,0.505882,0.0,0.136905,1823.0,-0.027761
4,0.101545,0.0,0.482858,0.333333,1.0,0.374153,0.147661,0.333333,8.0,0.6,0.000319,0.008403,0.435294,0.0,0.198413,1823.0,-0.027761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661595,0.000000,0.0,0.033372,0.000000,0.0,0.608278,0.097098,0.000000,0.0,0.0,0.004110,0.029978,0.647059,0.0,0.146825,8438.0,-0.029193
661596,0.000000,0.0,0.070786,0.000000,0.0,0.619473,0.139931,0.000000,0.0,0.0,0.001175,0.046879,0.623529,0.0,0.099206,8438.0,-0.029193
661597,0.000000,0.0,0.106114,0.000000,0.0,0.569125,0.087680,0.000000,0.0,0.0,0.001327,0.019891,0.470588,0.0,0.107143,8438.0,-0.029193
661598,0.000000,0.0,0.150828,0.000000,0.0,0.463550,0.090001,0.000000,0.0,0.0,0.007807,0.024678,0.400000,0.0,0.172619,8438.0,-0.029193


In [48]:
X_Y.to_csv(r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Safran\X_Y.csv')